In [0]:
from google.colab import drive
drive.mount("/content/gdrive")

In [0]:
from joblib import dump, load
mlp = load('gdrive/My Drive/Colab/News Subjectivity/neural_classifier.joblib') 

In [0]:
!pip install gluonnlp
!pip install mxnet
!pip install -U sentence-transformers

In [0]:
import gluonnlp as nlp
import numpy as np

In [0]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('bert-base-nli-mean-tokens')

In [0]:
import glob, os
p_labels = []
for file in sorted(glob.glob("gdrive/My Drive/Colab/News Subjectivity/test_sentence/*.txt")):
  news_dataset_test = nlp.data.TSVDataset(file, num_discard_samples=1)
  check = True
  for i in range(len(news_dataset_test)):
    if 'I' in news_dataset_test[i] or news_dataset_test[i][1] == '':
      continue
    else:
      character_vector = model.encode([news_dataset_test[i][0]])[0]
      character_vector = np.expand_dims(character_vector, axis=0)
      if check:
        char_vectors_test = character_vector.copy()
        check = False
      else:
        char_vectors_test = np.concatenate((char_vectors_test, character_vector), axis=0)
  p_labels.append(np.mean(mlp.predict(char_vectors_test).astype(int)))

In [0]:
labels_test = []
check = True
news_dataset_test_labels = nlp.data.TSVDataset('gdrive/My Drive/Colab/News Subjectivity/test_body.txt')
for i in range(len(news_dataset_test_labels)):
  if 'I' in news_dataset_test_labels[i] or news_dataset_test_labels[i][1] == '':
    continue
  else:
    if news_dataset_test_labels[i][1] == '':
      print(news_dataset_test_labels[i])
    labels_test.append(news_dataset_test_labels[i][12].strip())

In [0]:
for i, e in enumerate(labels_test):
  if int(e) > 2:
    labels_test[i] = str(2)

In [167]:
np.array(labels_test).astype(int)

array([2, 2, 1, 1, 1, 2, 2, 1, 2, 2, 2, 2, 1, 1, 2, 1, 2, 2, 2, 1, 1, 2,
       2, 2, 1, 2, 2, 2, 2, 1, 1, 2, 2, 2, 1, 2, 1, 2, 1, 1, 2, 1, 2, 2,
       2, 2, 2, 2, 1, 1, 2, 1, 2, 2, 1, 1, 2, 1, 2, 1, 1, 2, 2, 1, 2, 2])

In [168]:
np.round(p_labels).astype(int)

array([2, 2, 2, 1, 2, 2, 1, 1, 2, 1, 2, 1, 1, 1, 2, 2, 2, 2, 2, 2, 1, 2,
       1, 2, 1, 2, 2, 2, 1, 1, 2, 2, 1, 1, 2, 2, 2, 1, 1, 1, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 1, 2, 1, 2, 1, 2, 1, 2, 2, 2, 1, 1, 2, 2, 2, 1, 2])

In [172]:
np.round(100*(1 - np.sum(np.abs(np.array(labels_test).astype(int) - np.round(p_labels).astype(int)))/len(p_labels)),2)

66.67

In [178]:
np.sum(np.abs(np.array(labels_test).astype(float) - np.array(p_labels)))/len(labels_test)

0.38734088835571534